In [1]:
import re
import pickle
from collections import defaultdict
from tqdm import tqdm

In [10]:
def extract_tuple(chunked_sent):    # 根据文本中的所有三元组
    
    collector = []
    for words, tags in chunked_sent:
        
        tri_tuple, raw_sent, is_use = element_extractor(words, tags)
        if is_use:
            collector.append((tri_tuple, raw_sent))
    
    return collector
    
def element_extractor(words, tags):    # 提取三元组
    
    first_container = []
    second_container = []
    third_container = []
    record_switch = 0 # 开关，判断是否录入
                       #0: 未录入
                       #0.5:第一元组录入中
                       #1:第一元组录入完成，第二元组未录入
                       #1.5:第二元组录入中
                       #2:第二元组录入完成，第三元组未录入
                       #2.5:第三元组录入中
                       #3:完成录入
    for pos, (w, t) in enumerate(zip(words, tags)):
        try:
            if t == "B-NP":
                if record_switch == 0:
                    first_container.append(w)
                    if pos < (len(tags) - 1):
                        if tags[pos+1] == "I-NP":
                            record_switch = 0.5
                        else:
                            record_switch = 1
                    else:
                        record_switch = 3

                if record_switch == 2:
                    third_container.append(w)
                    if pos < (len(tags) - 1):
                        if tags[pos+1] == "I-NP":
                            record_switch = 2.5
                        else:
                            record_switch = 3
                    else:
                        record_switch = 3

            if t == "I-NP":
                if record_switch == 0.5:
                    first_container.append(w)
                    if pos < (len(tags) - 1):
                        if tags[pos+1] != "I-NP":
                            record_switch = 1
                    else:
                        record_switch = 3

                if record_switch == 2.5:
                    third_container.append(w)
                    if pos < (len(tags) - 1):
                        if tags[pos+1] != "I-NP":
                            record_switch = 3
                    else:
                        record_switch = 3

            if t == "B-VP":
                if record_switch == 1:
                    second_container.append(w)
                    if pos < (len(tags) - 1):
                        if tags[pos+1] == "I-VP":
                            record_switch = 1.5
                        else:
                            record_switch = 2
                    else:
                        record_switch = 3

            if t == "I-VP":
                if record_switch == 1.5:
                    second_container.append(w)
                    if pos < (len(tags) - 1):
                        if tags[pos+1] != "I-VP":
                            record_switch = 2
                    else:
                        record_switch = 3
        except IndexError:
            print(pos, len(words), len(tags))
                    
    if (len(first_container) == 0) or (len(second_container) == 0) or (len(third_container) == 0):
        return None, None, False
    
    joined_words = " ".join(words)
    joined_words = re.sub(r'\s([\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）](?:\s|$))', r'\1', joined_words)
    
    return (first_container, second_container, third_container), joined_words, True

In [11]:
with open('../datas/tuple/data_tagger_after_lstmcrf.pkl', 'rb') as bf:
    raw_iob_text = pickle.load(bf)
    
tuple_dict = defaultdict(list)
for key, value in tqdm(raw_iob_text.items()):
    tp = extract_tuple(value)
    if len(tp) > 0:
        tuple_dict[key].extend(tp)

100%|██████████| 15907/15907 [01:43<00:00, 153.49it/s]


In [13]:
# 二进制形式保存
with open('../datas/tuple/wiki_tri_tuple.pkl', 'wb') as bfile:
    pickle.dump(tuple_dict, bfile, protocol=4)